Min support of 0.05 was used to build the frequent itemsets and confidence more than 70% was considered to build the recommendation rules.

# Apriori algo is performed for recommendation. Since dataset is big, only customers who has watched more than 15 programs and is not inclined towards cricket(% of their total time is less than 70% for cricket) are chosen for the model

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import seaborn as sns
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

df_org = pd.read_csv('/content/drive/MyDrive/data_with_cluster.csv',index_col=0) 

In [ ]:
# for non cricket genre
df_org.drop(df_org[df_org['Cricket']>70].index,inplace=True)
df=df_org[(df_org['no_pgm_watched']>=15)]
df.reset_index(drop=True,inplace=True)
df.head()

,ID,genres,titles,cities,segment,dow,tod,prime_time,prime_day,prefered_genre,gen,Drama,Cricket,Others,Family,Crime,Romance,Action,Comedy,LiveTV,TalkShow,Reality,Thriller,Sport,Kids,Mythology,%time_prefered_genre,no_place_presence,%watched_base_loc,most_watched_pgm,%watched_fav_pgm,no_pgm_watched,total_watch_time,cluster,clust
0,train-121673,"Drama:48949,Family:9927,Crime:1901,Romance:165...","Naamkarann:369,Ishq:59,Pardes Mein Hai Mera Di...","ahmedabad:56666,mumbai:26989",neg,"1:3257,3:9162,2:12893,5:9237,4:8503,7:25094,6:...","11:7726,10:3521,13:2067,12:3844,20:4608,21:936...",2.0,7.0,Drama,"Drama:48949,Family:9927,Crime:1901,Romance:165...",58.525533,0.000000,0.000000,11.869149,2.272917,19.813001,2.467807,5.051592,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,58.525533,2.0,67.737732,Ghulaam,50.893306,33.0,83635.0,13,2
1,train-121670,"Cricket:5715,Family:2777,Drama:20873,LiveTV:4,...","India vs Australia 1st Test Hindi:83,India vs ...",navi mumbai:29400,neg,"1:40,3:2700,2:4659,5:7338,4:6641,7:2218,6:5800","11:302,10:1594,13:331,12:133,20:3802,14:3083,2...",9.0,5.0,Drama,"Cricket:5715,Family:2777,Drama:20873,LiveTV:4,...",71.001429,19.440098,0.000000,9.446221,0.000000,0.000000,0.047622,0.051024,0.013606,0.000000,0.000000,0.000000,0.000000,0.0,0.0,71.001429,1.0,100.000000,Chandra Nandni,70.984861,17.0,29396.0,13,3
2,train-199734,"Romance:1889,Cricket:739,TalkShow:605,Wildlife...","Pink:19,Andaz Apna Apna:49,Rangoon:59,Savdhaan...","kolkata:16889,delhi:180,chennai:245",neg,"1:275,3:1090,2:4390,5:25,4:2796,6:8732","11:281,10:775,13:735,15:213,21:225,22:2907,23:...",1.0,6.0,Drama,"Romance:1889,Cricket:739,TalkShow:605,Others:5...",56.187890,4.269702,0.300439,0.000000,20.360527,10.914028,0.993760,3.478160,0.000000,3.495493,0.000000,0.000000,0.000000,0.0,0.0,56.187890,3.0,97.545339,Jolly LLB,41.846758,24.0,17305.0,13,3
3,train-123297,"Drama:6551,Cricket:15808,Football:160,Thriller...","Bajrangi Bhaijaan:5,2nd T20I: India Fall of Wi...","pune:17288,delhi:5139,gurgaon:12,mumbai:2111",neg,"1:311,2:980,5:1136,4:6330,7:9249,6:6538","11:449,13:2123,15:558,14:2304,17:2328,16:4585,...",16.0,7.0,Cricket,"Drama:6551,Cricket:15808,Sport:160,Thriller:3,...",26.688666,64.401532,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.245743,0.012222,0.651837,0.0,0.0,64.401532,4.0,70.419552,India vs England | 2nd ODI | English,22.352174,30.0,24543.0,5,2
4,train-123299,"Drama:13815,Family:108423,Romance:46093,LiveTV...","Jekhane Bhooter Bhoy:11,Pardes Mein Hai Mera D...","delhi:1286,navi mumbai:202723,mumbai:1508",pos,"1:28375,3:35336,2:20357,5:28410,4:24327,7:4077...","11:6158,10:13943,13:9967,12:7376,15:24243,14:1...",15.0,7.0,Family,"Drama:13815,Family:108423,Romance:46093,LiveTV...",6.722431,0.000000,0.000000,52.759044,0.000000,22.429029,0.237949,0.000000,0.114352,0.000000,0.000000,17.737195,0.000000,0.0,0.0,52.759044,3.0,98.640502,Ke Apon Ke Por,51.806127,22.0,205503.0,1,0


In [ ]:
df=df[['ID','titles']]

In [ ]:
df.shape

(25977, 2)

In [ ]:
df['title_new']=np.NaN
df['duration']=np.NaN

In [ ]:
df['title_new'] = df['title_new'].astype('object')
df['duration'] = df['duration'].astype('object')

In [ ]:
for j in range(0,len(df)):
    y= len(df.loc[j,'titles'].split(","))
    l=[]
    k=[]
    for i in range(0, y):
        x=df.loc[j,'titles'].split(",")[i].rsplit(":",1)
        l.append(int(x[1]))
        k.append((x[0]))
    
    df.at[j,'title_new']=k  
    df.at[j,'duration']=l
df    

,ID,titles,title_new,duration
0,train-121673,"Naamkarann:369,Ishq:59,Pardes Mein Hai Mera Di...","[Naamkarann, Ishq, Pardes Mein Hai Mera Dil, S...","[369, 59, 5863, 223, 1, 835, 16, 6, 749, 936, ..."
1,train-121670,"India vs Australia 1st Test Hindi:83,India vs ...","[India vs Australia 1st Test Hindi, India vs A...","[83, 446, 4, 20866, 1, 616, 98, 200, 80, 4, 27..."
2,train-199734,"Pink:19,Andaz Apna Apna:49,Rangoon:59,Savdhaan...","[Pink, Andaz Apna Apna, Rangoon, Savdhaan Indi...","[19, 49, 59, 619, 658, 172, 182, 5, 32, 605, 4..."
3,train-123297,"Bajrangi Bhaijaan:5,2nd T20I: India Fall of Wi...","[Bajrangi Bhaijaan, 2nd T20I: India Fall of Wi...","[5, 3, 41, 18, 116, 449, 4, 25, 0, 174, 5485, ..."
4,train-123299,"Jekhane Bhooter Bhoy:11,Pardes Mein Hai Mera D...","[Jekhane Bhooter Bhoy, Pardes Mein Hai Mera Di...","[11, 21, 13, 9753, 6, 10937, 1, 1507, 503, 106..."
...,...,...,...,...
25972,train-90195,"Naamkarann:1295,Iss Pyar Ko Kya Naam Doon:10,D...","[Naamkarann, Iss Pyar Ko Kya Naam Doon, Dhoni'...","[1295, 10, 78, 8303, 524, 56, 2460, 9623, 47, ..."
25973,train-90196,"Naamkarann:2668,Chandra Nandni:2650,Pardes Mei...","[Naamkarann, Chandra Nandni, Pardes Mein Hai M...","[2668, 2650, 2792, 11849, 342, 26096, 16761, 9..."
25974,train-29508,"Bedroom Partner:5,10 Endrathukulla:4510,RPS vs...","[Bedroom Partner, 10 Endrathukulla, RPS vs DD,...","[5, 4510, 37, 6, 162, 1071, 494, 6, 4394, 922,..."
25975,train-126320,"Aus 1st Innings: Fall of Wickets:359,Day 4: In...","[Aus 1st Innings: Fall of Wickets, Day 4: Indi...","[359, 5, 14, 108, 81, 17240, 65, 328, 79, 153,..."


In [ ]:
df.head()

,ID,titles,title_new,duration
0,train-121673,"Naamkarann:369,Ishq:59,Pardes Mein Hai Mera Di...","[Naamkarann, Ishq, Pardes Mein Hai Mera Dil, S...","[369, 59, 5863, 223, 1, 835, 16, 6, 749, 936, ..."
1,train-121670,"India vs Australia 1st Test Hindi:83,India vs ...","[India vs Australia 1st Test Hindi, India vs A...","[83, 446, 4, 20866, 1, 616, 98, 200, 80, 4, 27..."
2,train-199734,"Pink:19,Andaz Apna Apna:49,Rangoon:59,Savdhaan...","[Pink, Andaz Apna Apna, Rangoon, Savdhaan Indi...","[19, 49, 59, 619, 658, 172, 182, 5, 32, 605, 4..."
3,train-123297,"Bajrangi Bhaijaan:5,2nd T20I: India Fall of Wi...","[Bajrangi Bhaijaan, 2nd T20I: India Fall of Wi...","[5, 3, 41, 18, 116, 449, 4, 25, 0, 174, 5485, ..."
4,train-123299,"Jekhane Bhooter Bhoy:11,Pardes Mein Hai Mera D...","[Jekhane Bhooter Bhoy, Pardes Mein Hai Mera Di...","[11, 21, 13, 9753, 6, 10937, 1, 1507, 503, 106..."


In [ ]:
data=[]
cols=['ID','Titles','Duration']
for j in range(0,len(df)):
  for i in range (0, len(df.loc[j,'title_new'])):
    data.append([df.loc[j,'ID'],df.loc[j,'title_new'][i],df.loc[j,'duration'][i]])  
df1=pd.DataFrame(data,columns=cols)

In [ ]:
df1.shape

(788032, 3)

In [ ]:
df1

,ID,Titles,Duration
0,train-121673,Naamkarann,369
1,train-121673,Ishq,59
2,train-121673,Pardes Mein Hai Mera Dil,5863
3,train-121673,Sasural Genda Phool,223
4,train-121673,Nagarjun Ek Yoddha,1
...,...,...,...
788027,train-126328,SRH Opening Ceremony,1879
788028,train-126328,India vs Australia 4th Test English,2635
788029,train-126328,Star Screen Awards,640
788030,train-126328,KXIP vs RPS,684


In [ ]:
df1['Titles'].nunique()

11074

In [ ]:
basket = (df1
          .groupby(['ID', 'Titles'])['Duration']
          .sum().unstack().reset_index().fillna(0)
          .set_index('ID'))

In [ ]:
basket.head()

Titles,44 - Finch Drills Boult,AB TO Z every shot in the book!,Ashwin's 3-68 Keeps India Alive,Australia vs India 1st Test: Damn! Kohli drop,Australia vs India 1st Test: Damn! Kohli drops a catch,Australia vs India 1st Test: Smith remembers,Australia vs India 1st Test: Warner gets emot,Australia vs India 1st Test: Warner gets emotional after reaching century,Baz pulls off a 'Dilscoop',Bhuvi's 3-44 Seals RR's Fate,Big Score was Around the Corner: Yuvi,Big score was around the corner: Yuvi,Billings Roy Hamper Pandya,Billings Roy Punish Pandya,Captains Introduce Teams,Day 2: Ashwin-Jadeja Prop up India,Duminy goes 6 6 on Bumrah,Early Kohli Exit Buoys England,England vs Australia One-off T20 International: Smith races to his fifty,Fan Interrupts Play to Greet Dhoni,Filmfare Awards South-Malayalam,GW 23 HLS: Liverpool 1-1 Chelsea,Gameplan: Can India Trump Australia?,Gony Goes for Plenty on Comeback,Haddin's Screamer Dismisses Gotch,Healy's Responsible 40 in the Final,Hingis Wins Swells Royals' Lead,IPL 'Spirit of Cricket',India Celebrate Resounding Series Win,Iss Pyar Ko Kya Naam Doon Ek Baar Phir,Jordan Chips in With 3-for,Jordan's 3 for 22 Flattens India,KP Faf Clobber Axar,Kabaddi Heroes: Rahul Chaudhari,Kohli Falls Cheaply Again,Kohli done in by Jubair's wrong 'un,Krunal's Steady 37* Steers MI Home,Kuldeep 3-for Stuns Oz,LOL! This Dhawan blooper,Li/Liu Beat Goh/Tan Enter Semis,...,Yuvraj's Late Blitz Goes in Vain,Yuvraj's Quick-fire 44 Helps SRH to 162,Yuvraj: The Scoring Machine,Zaheer Ends Martin Guptill's Stay,Zaheer Khan: Master of the Dark Arts,Zaheer Nips out Chris Gayle for a Duck,Zaheer Rewinds Time with 3-20,Zaheer Still a Young Gun,Zaheer Turns Back Time Against KKR,Zahreeley,Zak Gets the Big Fish ABD Falls!,Zakkariyayuda Gharbhinikal,Zakkariyayuda Gharbhinikal,Zale Mokale Aakash,Zameen,Zampa Celebrates Return with 3-for,Zampa Double Strike Rocks SL,Zampa Hits SRH for a Six,Zampa Zaps the Zing Out of RCB Chase,Zampa's 3-25 Chokes SL,Zampa's Football-Style Header Runs Nevill Out!,Zed Plus,Zeher The Revenge,Zhang/Li Overpower Lu/Huang,Zhang/Li Win Mixed Doubles Final,Zid,Zindagi,Zindagi Abhi Baki Hai Mere Ghost,Zindagi Na Milegi Dobara,Zlatan directs perfect goal,Zootopia,Zulfiqar,Zulmi,choker tara tui,match_highlights,match_replay,replay,siya ke ram,vivo IPL,www_Bhaya_com
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train-100007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
train-100015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
train-100023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
train-100050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
train-100054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [ ]:
basket_sets = basket.applymap(encode_units)

In [ ]:
basket_sets.head()

Titles,44 - Finch Drills Boult,AB TO Z every shot in the book!,Ashwin's 3-68 Keeps India Alive,Australia vs India 1st Test: Damn! Kohli drop,Australia vs India 1st Test: Damn! Kohli drops a catch,Australia vs India 1st Test: Smith remembers,Australia vs India 1st Test: Warner gets emot,Australia vs India 1st Test: Warner gets emotional after reaching century,Baz pulls off a 'Dilscoop',Bhuvi's 3-44 Seals RR's Fate,Big Score was Around the Corner: Yuvi,Big score was around the corner: Yuvi,Billings Roy Hamper Pandya,Billings Roy Punish Pandya,Captains Introduce Teams,Day 2: Ashwin-Jadeja Prop up India,Duminy goes 6 6 on Bumrah,Early Kohli Exit Buoys England,England vs Australia One-off T20 International: Smith races to his fifty,Fan Interrupts Play to Greet Dhoni,Filmfare Awards South-Malayalam,GW 23 HLS: Liverpool 1-1 Chelsea,Gameplan: Can India Trump Australia?,Gony Goes for Plenty on Comeback,Haddin's Screamer Dismisses Gotch,Healy's Responsible 40 in the Final,Hingis Wins Swells Royals' Lead,IPL 'Spirit of Cricket',India Celebrate Resounding Series Win,Iss Pyar Ko Kya Naam Doon Ek Baar Phir,Jordan Chips in With 3-for,Jordan's 3 for 22 Flattens India,KP Faf Clobber Axar,Kabaddi Heroes: Rahul Chaudhari,Kohli Falls Cheaply Again,Kohli done in by Jubair's wrong 'un,Krunal's Steady 37* Steers MI Home,Kuldeep 3-for Stuns Oz,LOL! This Dhawan blooper,Li/Liu Beat Goh/Tan Enter Semis,...,Yuvraj's Late Blitz Goes in Vain,Yuvraj's Quick-fire 44 Helps SRH to 162,Yuvraj: The Scoring Machine,Zaheer Ends Martin Guptill's Stay,Zaheer Khan: Master of the Dark Arts,Zaheer Nips out Chris Gayle for a Duck,Zaheer Rewinds Time with 3-20,Zaheer Still a Young Gun,Zaheer Turns Back Time Against KKR,Zahreeley,Zak Gets the Big Fish ABD Falls!,Zakkariyayuda Gharbhinikal,Zakkariyayuda Gharbhinikal,Zale Mokale Aakash,Zameen,Zampa Celebrates Return with 3-for,Zampa Double Strike Rocks SL,Zampa Hits SRH for a Six,Zampa Zaps the Zing Out of RCB Chase,Zampa's 3-25 Chokes SL,Zampa's Football-Style Header Runs Nevill Out!,Zed Plus,Zeher The Revenge,Zhang/Li Overpower Lu/Huang,Zhang/Li Win Mixed Doubles Final,Zid,Zindagi,Zindagi Abhi Baki Hai Mere Ghost,Zindagi Na Milegi Dobara,Zlatan directs perfect goal,Zootopia,Zulfiqar,Zulmi,choker tara tui,match_highlights,match_replay,replay,siya ke ram,vivo IPL,www_Bhaya_com
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train-100007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
train-100015,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
train-100023,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
train-100050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
train-100054,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
frequent_itemsets1 = apriori(basket_sets, min_support=0.05, use_colnames=True)

In [ ]:
frequent_itemsets1.head()

,support,itemsets
0,0.083497,(Aus 1st Innings: Fall of Wickets)
1,0.055241,(Aus Turn the Tide to Humble India)
2,0.052008,(Baahubali - The Beginning)
3,0.050699,(Bol Viru Bol)
4,0.252377,(Chandra Nandni)


In [ ]:
frequent_itemsets1.shape

(918, 2)

In [ ]:
rules1 = association_rules(frequent_itemsets1, metric="confidence", min_threshold=0.7)
rules1.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Ghulaam),(Chandra Nandni),0.155291,0.252377,0.110405,0.710957,2.817042,0.071213,2.586544
1,(Mere Angne Mein),(Chandra Nandni),0.110867,0.252377,0.084113,0.758681,3.006139,0.056132,3.098063
2,(Naamkarann),(Chandra Nandni),0.214228,0.252377,0.154675,0.722013,2.860848,0.100609,2.689413
3,(Chandra Nandni),(Yeh Rishta Kya Kehlata Hai),0.252377,0.293914,0.181160,0.717816,2.442266,0.106983,2.502217
4,(Day 4: India Move in for the Kill),(India vs Australia 3rd Test English),0.080918,0.253763,0.057743,0.713606,2.812097,0.037209,2.605632


In [ ]:
rules1.shape

(2348, 9)

In [ ]:
rules1.to_csv("rules.csv")